In [ ]:
!rm -r 'IncrementalLeraningMLDL'
!git clone "https://github.com/wAnto97/IncrementalLeraningMLDL"
from IncrementalLeraningMLDL.src.CIFAR100_dataset import MyCIFAR100
from IncrementalLeraningMLDL.src.Utils import Utils
from IncrementalLeraningMLDL.src.MyNet import MyNet
from IncrementalLeraningMLDL.src.KNN import KNN
from IncrementalLeraningMLDL.src.Exemplars import Exemplars
from IncrementalLeraningMLDL.src.Loss import Loss

import numpy as np
import sys
import copy
from torch.backends import cudnn
from torchvision import transforms
import torch
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from torch.utils.data import  DataLoader

from google.colab import drive
drive.mount('/content/gdrive')

Cloning into 'IncrementalLeraningMLDL'...
remote: Enumerating objects: 140, done.
remote: Counting objects: 100% (140/140), done.
remote: Compressing objects: 100% (86/86), done.
remote: Total 1324 (delta 93), reused 98 (delta 53), pack-reused 1184
Receiving objects: 100% (1324/1324), 10.86 MiB | 21.46 MiB/s, done.
Resolving deltas: 100% (894/894), done.
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


**Loading data**

In [ ]:
train_transform = transforms.Compose([
                                      transforms.RandomCrop(32, padding=4),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(), # Turn PIL Image to torch.Tensor
                                       transforms.Normalize( (0.4914, 0.4822, 0.4465),(0.2023, 0.1994, 0.2010))]) # Normalizes tensor with mean and standard deviation
                                      #transforms.Normalize( (0.5, 0.5, 0.5),(0.5, 0.5, 0.5))]) # Normalizes tensor with mean and standard deviation


# Define transforms for the evaluation phase
eval_transform = transforms.Compose([
                                      transforms.ToTensor(),
                                      transforms.Normalize( (0.4914, 0.4822, 0.4465),(0.2023, 0.1994, 0.2010))])  
training_set = MyCIFAR100('/content',train=True, n_groups=10, transform=train_transform, download=True,random_state = 653)
test_set = MyCIFAR100('/content',train=False, n_groups=10, transform=eval_transform, download=True,random_state = 653)

Files already downloaded and verified
Files already downloaded and verified


**Hyperparameters**

In [ ]:
DEVICE = 'cuda' # 'cuda' or 'cpu'  

BATCH_SIZE = 128      # Higher batch sizes allows for larger learning rates. An empirical heuristic suggests that, when changing
                     # the batch strain_dataloaderize, learning rate should change by the same factor to have comparable results
LR = 2              # The initial Learning Rate
MOMENTUM = 0.9       # Hyperparameter for SGD, keep this at 0.9 when using SGD
WEIGHT_DECAY = 1e-5  # Regularization, you can keep this at the default

NUM_EPOCHS = 70             # Total number of training epochs (iterations over dataset)
STEP_SIZE = [49,63]      # How many epochs before decreasing learning rate (if using a step-down policy)
GAMMA = 0.2                 # Multiplicative factor for learning rate step-down

LOG_FREQUENCY = 10

CLASSES_PER_GROUP=10
NUM_GROUPS=10
NUM_EXEMPLARS=2000

**Utils functions**

In [ ]:
def validation(val_dataloader,knn,conf_matrix=False):
    running_corrects = 0
    y_pred = []
    all_labels = []
    for images, labels,_ in val_dataloader:

        images = images.to(DEVICE)
        labels = labels.to(DEVICE)

        # Get predictions
        preds = knn.classify(images)
        # Update Corrects

        running_corrects += torch.sum(preds == labels.data).data.item()
        y_pred += list(map(lambda x : x.item(),preds))
        all_labels += list(labels)

        # Calculate Accuracy
    accuracy = running_corrects / float(len(val_dataloader.dataset))

    if(conf_matrix == True):
        all_labels = list(map(lambda label : label.item(),all_labels))
        return accuracy,confusion_matrix(y_pred,np.array(all_labels))

    return accuracy

In [ ]:
def test_loaders(train_dataloader):
    all_labels = []
    for images,labels,_ in train_dataloader:
      all_labels += list(map(lambda x: x.item(),labels))

    print(np.unique(all_labels,return_counts=True))

In [ ]:
def after_training(step,n_old_classes,exemplar_dataloader,train_dataloader_exemplars,exemplars,knn,net,utils,training_set,type_reduction = 'random'):
  
  images_indices = utils.create_images_indices(exemplar_dataloader,step)
  knn.update(net,train_dataloader_exemplars)  
  
  if len(exemplars.exemplar_set) > 0:
    print("Reducing the exemplar set..")
    exemplars.reduce_exemplars(n_old_classes)
  
  print("Building the exemplar set...")
  if type_reduction == 'random':
    exemplars.build_exemplars_random(images_indices,n_old_classes)
  elif type_reduction == 'herding':
    exemplars.build_exemplars_herding(net,images_indices,n_old_classes)

  return 

**Main**

In [ ]:
myNet = MyNet(n_classes=CLASSES_PER_GROUP)
knn = KNN()
exemplars = Exemplars()
utils = Utils()
myLoss = Loss()
typeScheduler='multistep' # In this case it can be only set to multistep

#Creating dataloader for the first group of 10 classes
train_dataloader_exemplars,test_dataloader = utils.create_dataloaders_icarl(training_set,test_set,1,exemplars.exemplar_set,BATCH_SIZE)

old_outputs=[]

for i in range(NUM_GROUPS):
    step=i+1 
    print("STARTING KNN TRAINING WITH GROUP:\t",step)  

    n_old_classes = CLASSES_PER_GROUP*(step-1)
    if step > 1:
      myNet.update_network(myNet.net,CLASSES_PER_GROUP + n_old_classes,myNet.init_weights)
      train_dataloader_exemplars,test_dataloader = utils.create_dataloaders_icarl(training_set,test_set,step,exemplars.exemplar_set,BATCH_SIZE)
      test_loaders(train_dataloader_exemplars)
    
    optimizer,scheduler = myNet.prepare_training(LR,MOMENTUM,WEIGHT_DECAY,STEP_SIZE,GAMMA,typeScheduler=typeScheduler)

    classification_losses = []
    distillation_losses = []

    myNet.net.to(DEVICE)
    cudnn.benchmark 

    for epoch in range(1):
        running_correct_train = 0
        if typeScheduler == 'multistep':
          print('Starting epoch {}/{}, LR = {}'.format(epoch+1, NUM_EPOCHS, scheduler.get_last_lr()))

        myNet.net.train() # Set Network to train mode
        current_step = 0
        for images, labels, _ in train_dataloader_exemplars:
            images = images.to(DEVICE)
            labels = labels.to(DEVICE)
            
            #Set all gradients to zero
            optimizer.zero_grad() 

            #Computing output and creating the acyclic graph for updating the gradients
            outputs = myNet.net(images) 

            #Computing predictions
            _, preds = torch.max(outputs.data, 1)
            
            #Get predictions of the previous net
            if(step > 1):
                old_outputs = myNet.get_old_outputs(images,labels)


            #Computing loss
            loss,clf_loss,dist_loss = myLoss.icarl_loss(old_outputs,outputs,labels,step,current_step,utils,CLASSES_PER_GROUP)
            classification_losses.append(clf_loss.item())
            distillation_losses.append(dist_loss.item())

            #Calculate correct predictions
            running_correct_train += torch.sum(preds == labels.data).data.item()

            #Accumulate gradients
            loss.backward()  

            # Update weights based on accumulated gradients  
            optimizer.step()

            current_step += 1
        
        #Calculate training accuracy
        train_accuracy = running_correct_train/len(train_dataloader_exemplars.dataset)
        print("Accuracy on the training :\t",train_accuracy)

        if typeScheduler == 'multistep':
            scheduler.step()

    #Handling Exemplars
    exemplar_dataloader = DataLoader(training_set.get_group(step),batch_size=BATCH_SIZE,drop_last=False,num_workers=4)
    after_training(step,n_old_classes,exemplar_dataloader,train_dataloader_exemplars,exemplars,knn,myNet.net,utils,training_set,type_reduction='random')

    #Test
    test_accuracy,test_matrix = validation(test_dataloader,knn,conf_matrix=True)
    print("Accuracy on the test :\t",test_accuracy)

    #Writing on file    
    utils.writeOnFileMetrics('KNNMetrics.json', step, [train_accuracy,None,test_accuracy,test_matrix.tolist()])
    utils.writeOnFileLosses('KNNLosses.json', step, [classification_losses,distillation_losses])
    !cp  './KNNMetrics.json' './gdrive/My Drive/KNNMetrics.json'
    !cp  'KNNLosses.json' './gdrive/My Drive/KNNLosses.json'

STARTING KNN TRAINING WITH GROUP:	 1
Starting epoch 1/70, LR = [0.2]
Accuracy on the training :	 0.1428


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=13, p=2,
                     weights='uniform')
Building the exemplar set...
Build: 200
Accuracy on the test :	 0.351
STARTING KNN TRAINING WITH GROUP:	 2
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19]), array([196, 199, 198, 199, 200, 194, 198, 198, 194, 197, 494, 496, 494,
       492, 497, 496, 496, 494, 494, 486]))
Starting epoch 1/70, LR = [0.2]
Accuracy on the training :	 0.04528571428571428


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=13, p=2,
                     weights='uniform')
Reducing the exemplar set..
Reduced: 100
Building the exemplar set...
Build: 100
Accuracy on the test :	 0.2185
STARTING KNN TRAINING WITH GROUP:	 3
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]), array([ 98,  99,  99,  99, 100, 100, 100,  98,  99,  97,  99,  98,  98,
       100, 100,  98,  99, 100,  97,  99, 495, 497, 494, 495, 493, 492,
       492, 490, 497, 490]))
Starting epoch 1/70, LR = [0.2]
Accuracy on the training :	 0.03242857142857143


...

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')
Reducing the exemplar set..
Reduced: 25
Building the exemplar set...
Build: 25
Accuracy on the test :	 0.0415
STARTING KNN TRAINING WITH GROUP:	 9
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87, 88, 89]), array([ 25,  25,  25,  25,  24,  25,  24,  25,  24,  25,  25,  24,  25,
        24,  24,  23,  25,  25,  25,  23,  25,  25,  25,  25,  25,  25,
        25,  24,  24,  25,  25,  25,  24,  24,  25,  22,  25,  25,  25,
        24,  24,  25,  25,  24,  24,  25,  25

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=13, p=2,
                     weights='uniform')
Reducing the exemplar set..
Reduced: 22
Building the exemplar set...
Build: 22
Accuracy on the test :	 0.043555555555555556
STARTING KNN TRAINING WITH GROUP:	 10
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]), array([ 21,  22,  22,  22,  21,  21,  22,  22,  22,  22,  22,  22,  21,
        22,  21,  22,  22,  22,  22,  22,  22,  22,  22,  22,  22,  22,
        22,  22,  22,  22,  22,  22,  22,  22,  22,  22,  22,

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=13, p=2,
                     weights='uniform')
Reducing the exemplar set..
Reduced: 20
Building the exemplar set...
Build: 20
Accuracy on the test :	 0.0395
